# <span style="color:darkblue"> Lecture 20 - Connecting to SQL via Python </span>

<font size = "5">

In the past lectures:

- We worked directly in SQL
- Imported data
- Carried out basic dataset operations

In  this lecture we will run SQL from Python

- Combine the best of both!
- Learn about escape characters!

# <span style="color:darkblue"> I. Import Libraries and Data </span>


<font size = "5">

Install new libraries to connect to SQL from Python:

- psycopg2
- sqlalchemy

In [1]:
# With base Python, run the following commands in the terminal
# (i)     pip3 install psycopg2
# (ii)    pip3 install sqlalchemy
# If (i) doesn't work, instead try
#         pip3 install psycopg2-binary           OR
#         pip3 install psycopg2-binary --user

# For Anaconda: Open the "Anaconda Navigator" app and go the "Environment" tab
# Check whether "psycopg2", "sqlalchemy" appear in the "installed" panel
# Otherwise, search in the "Not installed" panel and install them

<font size = "5">

Import libraries

In [1]:
# psycogpg2 helps us process SQL commands to send to the server
# sqlalchemy facilitates establishing a connection with the server

import pandas as pd
import psycopg2
import sqlalchemy as sa
from   sqlalchemy.engine import URL
from   sqlalchemy import text

<font size = "5">

Import Data

In [2]:
# Import the two car racing datasets

results  = pd.read_csv("data_raw/results.csv")
races    = pd.read_csv("data_raw/races.csv")
circuits = pd.read_csv("data_raw/circuits.csv")

<font size = "5">

Connect to SQL server

In [3]:
# Use the connection details to your server
# These are the default settings.
# "postgresql" is a fixed argument
# If you have a different host,database, username, or password,
# change the corresponding connection details

url_server = URL.create(
    "postgresql",
    host = 'localhost',
    database = 'postgres',
    username = 'postgres',
    port = 5432,
    password  = '12345')

con = sa.create_engine(url_server).connect()


<font size = "5">

Upload data to SQL

In [4]:
# Import the "results" table, with the name "results_sql"
# Con is an argument to specify the server connection
# "if_exists" is an option to replace the table if it already exists
# You can choose any name instead of "results_sql"

results.to_sql('results_sql',
               con = con,
               if_exists='replace')

# Import "races"
races.to_sql('races_sql',con, if_exists='replace')

102

<font size = "5">

Try it yourself!

- Upload the "circuits table" into SQL <br>
using ".to_sql()"

In [5]:
# Write your own code

circuits.to_sql('circuits_sql',
                      con = con,
                      if_exists = 'replace')



77

# <span style="color:darkblue"> II. "Escape" Characters </span>

<font size = "5">

- Backslash (\\) is known as an escape character
- Used for special operations on strings
- Great for working with text!

<font size = "5">

Multi-line strings (" \ ")

In [6]:
# Use a backslash to define strins over multiple lines
# Do not include a space after "\", otherwise it won't work.
example_string = "This is a string \
                  defined over multiple lines"

<font size = "5">

"Double quotes" inside string

In [7]:
# Use a backslash + quotation 

example_double = "This will \"supposedly\" put double quotes inside a string"
print(example_double)

This will "supposedly" put double quotes inside a string


<font size = "5" >
'Single quotes' inside string

In [8]:
# There is no need for a backslash given single quotes 

example_single = "This will 'supposedly' put single quotes inside a string"
print(example_single)

This will 'supposedly' put single quotes inside a string


<font size = "5">

Try it yourself!

- Print a string ``` SELECT "driverId" FROM results; ``` <br> 
using backslash

In [10]:
# Write your own code

double_string = "SELECT \"driverId\" FROM results;"
print(double_string)




SELECT "driverId" FROM results;


# <span style="color:darkblue"> II. Data operations </span>

<font size = "5">

- In SQL, operations are often called "queries"


<font size = "5">

Import Data from SQL

In [11]:
# Use "pd.read_sql()"
# - The first argument is string with instructions wrapped in text()
#- The second argument is the server connection

# Extract all data from a column
example1 = pd.read_sql(text("SELECT * FROM results_sql;"),con)

# Extract a subset of columns
example2 = pd.read_sql(text("SELECT points \
                             FROM results_sql;"),con)

# Subset based on a string condition
example3 = pd.read_sql(text("SELECT * \
                             FROM races_sql \
                             WHERE name = 'Abu Dhabi Grand Prix';"),con)

# Note: (i) Remember to include "\"  to be able to define
#           strings over multiple lines
#       (ii) We can include single quotations in the WHERE command
#            without any additional escape characters

<font size = "5">

Upper case columns

- In SQL syntax we use double quotes e.g. ``` "driverId" ```
- ```.read_sql()``` requires a string inside a string
- To do so, use escape characters, e.g.   ``` \"driverId\" ```

In [12]:
# Select a column
example4 = pd.read_sql(text("SELECT \"driverId\" \
                             FROM results_sql;"),con)

# Compute an aggregate statistic
example5 = pd.read_sql(text("SELECT AVG(points) as mean_points \
                             FROM results_sql \
                             GROUP BY \"driverId\" ;"),con)


<font size = "5">

Merge two datasets

In [13]:
# Merge 
# Reference the column \"driverId\" with escape characters
example6 = pd.read_sql(text("SELECT * \
                             FROM results_sql \
                             LEFT JOIN races_sql \
                             ON results_sql.\"raceId\" = races_sql.\"raceId\" ;"),con)

# Merge a subset of columns
# Use "results_sql.*" to select all columns from the primary dataset
# Use "races_sql.date" to only select the "date" column from the secondary dataset 

example7 = pd.read_sql(text("SELECT results_sql.*, races_sql.date \
                             FROM results_sql \
                             LEFT JOIN races_sql \
                             ON results_sql.\"raceId\" = races_sql.\"raceId\" ;"),con)


<font size = "5">

Try it yourself!

- Practice the ```pd.read_sql()``` command
- FROM results_sql compute the sum of points by <br>
"raceId" <br>

In [15]:
# Write your own code

try_yourself = pd.read_sql(text("SELECT SUM(points) as SUM_points \
                             FROM results_sql \
                             GROUP BY \"raceId\" ;"),con)

try_yourself



,sum_points
0,25.0
1,26.0
2,39.0
3,101.0
4,101.0
...,...
1074,39.0
1075,39.0
1076,26.0
1077,24.0


<font size = "5">

Try it yourself!

- Practice the ```pd.read_sql()``` command
- Merge "races_sql" and the circuits table on "circuitId" <br>
tha you imported above using ```LEFT JOIN```

In [16]:
# Write your own

pd.read_sql(text("SELECT races_sql.*, circuits_sql.\"circuitId\" \
                             FROM races_sql \
                             LEFT JOIN circuits_sql \
                             ON races_sql.\"circuitId\" = circuits_sql.\"circuitId\" ;"),con)



,index,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time,circuitId
0,0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,1
1,1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,2
2,2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,17
3,3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,3
4,4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,1097,1116,2023,19,69,United States Grand Prix,2023-10-22,19:00:00,https://en.wikipedia.org/wiki/2023_United_Stat...,2023-10-20,17:30:00,2023-10-21,18:00:00,\N,\N,2023-10-20,21:00:00,2023-10-21,22:00:00,69
1098,1098,1117,2023,20,32,Mexico City Grand Prix,2023-10-29,20:00:00,https://en.wikipedia.org/wiki/2023_Mexico_City...,2023-10-27,18:30:00,2023-10-27,22:00:00,2023-10-28,17:30:00,2023-10-28,21:00:00,\N,\N,32
1099,1099,1118,2023,21,18,São Paulo Grand Prix,2023-11-05,17:00:00,https://en.wikipedia.org/wiki/2023_S%C3%A3o_Pa...,2023-11-03,14:30:00,2023-11-04,14:30:00,\N,\N,2023-11-03,18:00:00,2023-11-04,18:30:00,18
1100,1100,1119,2023,22,80,Las Vegas Grand Prix,2023-11-19,06:00:00,https://en.wikipedia.org/wiki/2023_Las_Vegas_G...,2023-11-17,04:30:00,2023-11-17,08:00:00,2023-11-18,04:30:00,2023-11-18,08:00:00,\N,\N,80


# <span style="color:darkblue"> IV. Additional Material </span>

<font size = "5">

More about SQL syntax

https://www.w3schools.com/sql/